In [7]:
import pandas as pd
import os
import re

So the goal is to see how similar my labels are to Marshall's. So I'll use 3D_Images_correct.csv, since it has every label for ADNI 1 that I will be using.

In [2]:
marshall_data = pd.read_excel(os.path.expanduser('~/Downloads/Profile Variables 2024-10-16.xlsx'))
my_data = pd.read_csv('../3D_Images_correct.csv')

#print(marshall_data.columns)

# Okay I have the data

In [3]:
marshall_data['DX_bl'].value_counts()

DX_bl
3    690
0    542
2    423
4    411
1    353
Name: count, dtype: int64

In [129]:
#marshall_data2 = marshall_data.copy()
marshall_data['Baseline'] = None
#marshall_data2['Baseline'] = None
# for i in range(len(marshall_data2)):
#     if marshall_data2.loc[i, 'DX_bl'] == 0:
#         marshall_data2.loc[i, 'Baseline'] = 'CN'
#     elif marshall_data2.loc[i, 'DX_bl'] == 1:
#        marshall_data2.loc[i, 'Baseline'] = 'SMC'
#     elif marshall_data2.loc[i, 'DX_bl'] == 2:
#         marshall_data2.loc[i, 'Baseline'] = 'EMCI'
#     elif marshall_data2.loc[i, 'DX_bl'] == 3:
#         marshall_data2.loc[i, 'Baseline'] = 'LMCI'
#     elif marshall_data2.loc[i, 'DX_bl'] == 4:
#         marshall_data2.loc[i, 'Baseline'] = 'AD'

for i in range(len(marshall_data)):
    if marshall_data.loc[i, 'DX_bl'] == 0:
        marshall_data.loc[i, 'Baseline'] = 'CN'
    elif marshall_data.loc[i, 'DX_bl'] == 1:
       marshall_data.loc[i, 'Baseline'] = 'SMC'
    elif marshall_data.loc[i, 'DX_bl'] == 2:
        marshall_data.loc[i, 'Baseline'] = 'MCI'
    elif marshall_data.loc[i, 'DX_bl'] == 3:
        marshall_data.loc[i, 'Baseline'] = 'MCI'
    elif marshall_data.loc[i, 'DX_bl'] == 4:
        marshall_data.loc[i, 'Baseline'] = 'AD'

In [66]:
print(marshall_data[['RID', 'DX_bl', 'Baseline']])

       RID  DX_bl Baseline
0        2      0       CN
1        3      4       AD
2        4      3     LMCI
3        5      0       CN
4        6      3     LMCI
...    ...    ...      ...
2414  7105      3     LMCI
2415  7114      1      SMC
2416  7117      2     EMCI
2417  7121      3     LMCI
2418  7123      1      SMC

[2419 rows x 3 columns]


In [24]:
my_data['RID'] = None
pattern = re.compile(r'ADNI_1\/ADNI\/(\d{3}_S_\d+)\/')

for i in range(len(my_data)):
    sid = re.search(pattern, my_data['File Path'][i])[1]
    rid = int(sid[-4:])
    my_data.loc[i, 'RID'] = rid

In [37]:
dallan = pd.read_csv('../3D_Images_dallan.csv')

In [38]:
dallan['RID'] = None
pattern = re.compile(r'ADNI_1\/ADNI\/(\d{3}_S_\d+)\/')

for i in range(len(dallan)):
    sid = re.search(pattern, dallan['File Path'][i])[1]
    rid = int(sid[-4:])
    dallan.loc[i, 'RID'] = rid

In [69]:
rgroup = []
unique_rids = my_data['RID'].unique()
for rid in unique_rids:
    label = my_data[my_data['RID'] == rid]['Research Group'].value_counts().index[0]
    rgroup.append(label)

In [79]:
x = pd.DataFrame({'RID':unique_rids,'Research Group':rgroup})

x.sort_values('RID',inplace=True)
x.reset_index(drop=True,inplace=True)

In [130]:
use_marshall = marshall_data[['RID', 'DX_bl', 'Baseline']]
use_marshall = use_marshall[use_marshall['RID'].isin(x['RID'].to_list())]

In [131]:
#x[~x['RID'].isin(use_marshall['RID'].to_list())]
use_x = x[x['RID'].isin(use_marshall['RID'].to_list())]

In [132]:
use_x.reset_index(drop=True,inplace=True)
use_marshall.reset_index(drop=True,inplace=True)

In [133]:
print(use_x.columns)
print(use_marshall.columns)

Index(['RID', 'Research Group'], dtype='object')
Index(['RID', 'DX_bl', 'Baseline'], dtype='object')


In [139]:
boolean = ~(use_x['Research Group'] == use_marshall['Baseline'])

use_x[boolean]

,RID,Research Group
0,2,MCI
284,492,MCI


In [144]:
print(my_data[my_data['RID'] == 492])

dallan[dallan['RID'] == 492]

                                              File Path Research Group  RID
8423  ADNI_1/ADNI/099_S_0492/MPRAGE_Repeat/2006-05-2...            MCI  492
8424  ADNI_1/ADNI/099_S_0492/MPRAGE/2006-05-25_08_01...             AD  492


,File Path,Research Group,RID
8423,ADNI_1/ADNI/099_S_0492/MPRAGE_Repeat/2006-05-2...,MCI,492
8424,ADNI_1/ADNI/099_S_0492/MPRAGE/2006-05-25_08_01...,MCI,492
